In [26]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

from fairness_checker import *

In [27]:
# To load the model back
with open('./model/adult_synth.pkl', 'rb') as f:
    model = pickle.load(f)

# Random Forest

In [46]:
with open('./model/adult_RF.pkl', 'rb') as f:
    clf = pickle.load(f)

In [47]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X.to_numpy())

    def preprocessing(self, df):
        df.drop(labels=["workclass", "education", "occupation", "relationship", "race", "native.country"], axis = 1, inplace = True)
        df.drop(labels=["income"], axis = 1, inplace = True)

        return df

trained = model_wrapper(clf)

In [49]:
# this takes 10 minutes
iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=30000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['income'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

In [48]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/adult_processed.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['income'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

In [50]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.1933447448 0.1082235768 0.0851211679 0.0014029132 1.7865307212
accuracy_eqaulity   0.0250249201 0.1391474896 0.1141225695 0.0029940212 0.1798445676
equalized_odds_1    0.0175621188 0.0875346615 0.0699725427 0.0016896592 0.2006304530
equalized_odds_2    0.0114542011 0.0624741778 0.0510199767 0.0101231894 0.1833429674
accuracy_equality_1 0.1941115666 0.1458089898 0.0483025769 0.0225247803 1.3312729684
accuracy_equality_2 0.1102330179 0.1394989392 0.0292659214 0.0066585384 0.7902068537
mean_difference     0.1801306340 0.1050887296 0.0750419043 0.0027038341 1.7140813728
sum of diff: 0.4728466593515629
avg of ratio: 0.8837014148906338


# Logistic Regression

In [33]:
with open('./model/adult_LR.pkl', 'rb') as f:
    clf = pickle.load(f)

In [34]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df.drop(labels=['income','native.country', 'hours.per.week'], axis = 1, inplace = True)
        scaler = StandardScaler()
        df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)


        return df

trained = model_wrapper(clf)

In [35]:
# takes 10 minutes
iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=30000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['income'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

In [36]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/adult_processed.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['income'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

In [37]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.0139228408 0.0284592240 0.0145363831 0.0028012685 0.4892206778
accuracy_eqaulity   0.0992683727 0.1092422431 0.0099738705 0.0027937253 0.9086995087
equalized_odds_1    0.0017247318 0.0119668633 0.0102421314 0.0024661815 0.1441256401
equalized_odds_2    0.0080650603 0.0183612086 0.0102961483 0.0063934861 0.4392445227
accuracy_equality_1 0.3459565225 0.1449367663 0.2010197562 0.0153266381 2.3869479862
accuracy_equality_2 0.1120956978 0.1466942950 0.0345985972 0.0041668844 0.7641449028
mean_difference     0.0678595096 0.0375924023 0.0302671072 0.0016941436 1.8051389460
sum of diff: 0.310933993918111
avg of ratio: 0.9910745977571719


# KNN

In [38]:
with open('./model/adult_KNN.pkl', 'rb') as f:
    clf = pickle.load(f)

In [39]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df = df[['age', 'workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']]
        scaler = StandardScaler()
        df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)


        return df

trained = model_wrapper(clf)

In [40]:
# takes 20 mins!
iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=30000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['income'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

In [41]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/adult_processed.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['income'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

In [42]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.1721440158 0.1046898005 0.0674542153 0.0047297522 1.6443246140
accuracy_eqaulity   0.0473392149 0.1176936877 0.0703544728 0.0037954066 0.4022239072
equalized_odds_1    0.0571962732 0.0786044106 0.0214081374 0.0048159543 0.7276471230
equalized_odds_2    0.1667534959 0.1239259158 0.0428275801 0.0076650720 1.3455901845
accuracy_equality_1 0.1009553523 0.1512929415 0.0503375892 0.0167325480 0.6672839543
accuracy_equality_2 0.1193507001 0.1448488999 0.0254981998 0.0090541438 0.8239669072
mean_difference     0.1654413724 0.1067693908 0.0586719816 0.0027132484 1.5495206177
sum of diff: 0.3365521762158183
avg of ratio: 1.022936758266558
